In [0]:
import pandas as pd
from fbprophet import Prophet 
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 

df = pd.read_csv('https://raw.githubusercontent.com/danielcaraway/data/master/Zip_Zhvi_SingleFamilyResidence.csv', encoding='latin')

In [5]:
to_drop = "1996-04	1996-05	1996-06	1996-07	1996-08	1996-09	1996-10	1996-11	1996-12".split()
df_97 = df.drop(to_drop, axis=1)
df_97_nona = df_97.dropna(subset=['1997-01'])
len(df_97_nona)

13916

In [0]:
df = df_97_nona.copy()
columns = df.columns[:7].values
region_reference = pd.DataFrame(data=df, columns=columns)
to_drop = ['RegionID',
 'City',
 'State',
 'Metro',
 'CountyName',
 'SizeRank']
just_numbers = df.drop(to_drop, axis=1)
df_t = just_numbers.set_index('RegionName').T
df_t.reset_index(inplace=True)
df_t['year'] = df_t.apply(lambda x: x['index'].split('-')[0], axis=1)
by_year = pd.DataFrame(df_t.groupby('year').mean())
by_year_t = by_year.reset_index()
by_year_t['year'] = by_year_t['year'].astype('datetime64[ns]') 
by_year_t.set_index('year', inplace=True)
by_year_t2 = by_year_t.T
by_year_t2.reset_index(inplace=True)

In [0]:
df = by_year_t2.copy()
df_sm = df[:5]
zip_table = []
def get_prophet_predictions(row):
  mini = pd.DataFrame(row)
  mini['ds'] = row.index
  mini['y'] = row.values
  df = mini.iloc[1:]

  m = Prophet()
  m.fit(df)
  invest_price = df.tail(1)
  future = m.make_future_dataframe(periods=60, freq='M')
  fcst = m.predict(future)
  sell_price = fcst.tail(1)

  roi = sell_price['trend'].values[0] - invest_price['y'].values[0]
  return pd.Series((fcst, roi))

In [10]:
df_sm[['forecast','roi']] = df_sm.apply(lambda x: get_prophet_predictions(x), axis=1)
# df[['forecast','roi']] = df.apply(lambda x: get_prophet_predictions(x), axis=1)

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 17.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 17.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 17.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run pr

In [11]:
# df.to_csv('prophet_df.csv')

year,RegionName,1997-01-01 00:00:00,1998-01-01 00:00:00,1999-01-01 00:00:00,2000-01-01 00:00:00,2001-01-01 00:00:00,2002-01-01 00:00:00,2003-01-01 00:00:00,2004-01-01 00:00:00,2005-01-01 00:00:00,2006-01-01 00:00:00,2007-01-01 00:00:00,2008-01-01 00:00:00,2009-01-01 00:00:00,2010-01-01 00:00:00,2011-01-01 00:00:00,2012-01-01 00:00:00,2013-01-01 00:00:00,2014-01-01 00:00:00,2015-01-01 00:00:00,2016-01-01 00:00:00,2017-01-01 00:00:00,2018-01-01 00:00:00,2019-01-01 00:00:00,forecast,roi
0,60657,378192.833333,389723.833333,453431.500000,521798.916667,586183.833333,628344.916667,6.691198e+05,7.073292e+05,7.562488e+05,8.070008e+05,8.102088e+05,8.231563e+05,7.720613e+05,746086.833333,709742.416667,710565.666667,791568.166667,8.466048e+05,8.935743e+05,9.266148e+05,9.397620e+05,9.536783e+05,9.566285e+05,ds trend ... multiplicat...,-243313.774765
1,77494,209368.250000,206195.250000,212468.750000,223155.583333,226383.916667,226765.000000,2.311428e+05,2.401042e+05,2.466654e+05,2.498909e+05,2.580278e+05,2.607339e+05,2.610133e+05,267443.583333,260171.666667,264811.166667,283496.833333,3.155041e+05,3.351682e+05,3.324876e+05,3.300152e+05,3.334704e+05,3.347144e+05,ds trend ... multiplicat...,-158187.740715
2,60614,573575.666667,616168.166667,706178.750000,804170.666667,905689.416667,983081.333333,1.042087e+06,1.086231e+06,1.164892e+06,1.234957e+06,1.246696e+06,1.179971e+06,1.070229e+06,984387.583333,909385.583333,868096.583333,959294.333333,1.032766e+06,1.095943e+06,1.142199e+06,1.175260e+06,1.194322e+06,1.190550e+06,ds trend ... multiplicat...,-722240.584607
3,77449,97162.916667,99462.916667,102394.750000,105518.250000,106366.250000,109551.000000,1.137417e+05,1.198598e+05,1.240328e+05,1.242390e+05,1.289552e+05,1.259122e+05,1.204772e+05,120319.166667,112696.750000,111168.416667,123927.750000,1.402042e+05,1.548712e+05,1.644619e+05,1.702064e+05,1.769127e+05,1.842622e+05,ds trend ... multiplicati...,-136073.190485
4,77084,96043.833333,97957.250000,101216.833333,104458.750000,105968.083333,109260.000000,1.127021e+05,1.193099e+05,1.231274e+05,1.239627e+05,1.281134e+05,1.262742e+05,1.232700e+05,122815.083333,115972.916667,114768.250000,125350.833333,1.396642e+05,1.535522e+05,1.617357e+05,1.673551e+05,1.741188e+05,1.823445e+05,ds trend ... multiplicati...,-135463.986688
